In [ ]:
import os
import shutil

# There are two ways to load the data from the PANDA dataset:
# Option 1: Load images using openslide

# Option 2: Load images using skimage (requires that tifffile is installed)
import skimage.io

# General packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import PIL
from IPython.display import Image, display
from collections import Counter

import cv2
import skimage.io
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
import tensorflow as tf


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip /content/gdrive/MyDrive/archive/archive.zip

Streaming output truncated to the last 5000 lines.
  inflating: train_label_masks/train_label_masks/88ecc79a10ff8a1f201b73b3e82a4648_mask.png  
  inflating: train_label_masks/train_label_masks/88ef695f58a55195f0730b4090583f4c_mask.png  
  inflating: train_label_masks/train_label_masks/88fb00fabc97d341a95e97a593c3fc43_mask.png  
  inflating: train_label_masks/train_label_masks/88fcb6de1ead418b238f198a635179c0_mask.png  
  inflating: train_label_masks/train_label_masks/88fd1b3540f148486d5987f4ba55b02f_mask.png  
  inflating: train_label_masks/train_label_masks/8900095e28a92eebb25f469b67d29b84_mask.png  
  inflating: train_label_masks/train_label_masks/890b1b3fc51fa2caf03d7bef25a31336_mask.png  
  inflating: train_label_masks/train_label_masks/890d0c582d769f001e95fa6f38252daf_mask.png  
  inflating: train_label_masks/train_label_masks/89159e7316ccc0d364591c421a5c7e56_mask.png  
  inflating: train_label_masks/train_label_masks/891f54c0f474191384a4b5e2d97b4696_mask.png  
  inflating: train_

In [ ]:
# Location of the training images
dataDir = '../content/train_images/train_images/'

# Location of training labels
trainLabels = pd.read_csv('/content/train.csv').set_index('image_id')
testDF = pd.read_csv('/content/test.csv').set_index('image_id')

# Output cropped images
#cropDir = '/kaggle/working/train_images/'

inputShape = (3, 224, 224)
epochs = 30

In [ ]:
# How many train objects should be included in one batch (higher = faster but less accurate)
# Take care that the batch size is smaller than the amount of total images analyzed
batch_size = 32
INIT_LR = 0.0001

In [ ]:
trainDatagen = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,height_shift_range=0.1, validation_split = 0.20,
                                  zoom_range=0.2, horizontal_flip=True, fill_mode="nearest")

In [ ]:
trainDF = pd.DataFrame(list(zip(trainLabels.index + ".png", trainLabels.isup_grade.astype(str))),
               columns =['x_col', 'y_col'])

# Uncomment the following if the assumption needs to be re-checked
# for x in trainDF.x_col:
#     assert x in os.listdir(dataDir)

In [ ]:
trainGenerator = trainDatagen.flow_from_dataframe(
    trainDF, x_col="x_col", y_col="y_col",
    directory=dataDir,  # this is the target directory
    batch_size=32,
    class_mode = "categorical",
    subset="training",
    target_size=(inputShape[0], inputShape[1]))

Found 8493 validated image filenames belonging to 6 classes.


In [ ]:
valGenerator = trainDatagen.flow_from_dataframe(
    trainDF, x_col="x_col", y_col="y_col",
    directory=dataDir,  # this is the target directory
    batch_size=32,
    class_mode = "categorical",
    subset="validation",
    target_size=(inputShape[0], inputShape[1]))

Found 2123 validated image filenames belonging to 6 classes.


# **VAN MODEL**

In [ ]:
!pip install transformers pillow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.6 MB/s eta 0:00:00


In [ ]:
from transformers import VanForImageClassification, VanConfig
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Location of the training images
dataDir = '../content/train_images/train_images/'

# Location of training labels
trainLabels = pd.read_csv('/content/train.csv').set_index('image_id')
testDF = pd.read_csv('/content/test.csv').set_index('image_id')

inputShape = (3,224, 224)
epochs = 30
batch_size = 32
INIT_LR = 0.0001

'''### Create an instance of the ImageDataGenerator
trainDatagen = ImageDataGenerator(
    rotation_range=30, width_shift_range=0.1, height_shift_range=0.1,
    zoom_range=0.2, horizontal_flip=True, fill_mode="nearest"
)

# Create a DataFrame for training
trainDF = pd.DataFrame(list(zip(trainLabels.index + ".png", trainLabels.isup_grade.astype(str))),
                       columns=['x_col', 'y_col'])

# Create data generators for training and validation
trainGenerator = trainDatagen.flow_from_dataframe(
    trainDF, x_col="x_col", y_col="y_col",
    directory=dataDir,
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
    target_size=(inputShape[0], inputShape[1])
)

valGenerator = trainDatagen.flow_from_dataframe(
    trainDF, x_col="x_col", y_col="y_col",
    directory=dataDir,
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
    target_size=(inputShape[0], inputShape[1])
) '''

# Load VAN-Small model
model_dir = "/content/"  # Change this path if you uploaded the files to a different directory
config = VanConfig.from_json_file(f"{model_dir}/config.json")
model = VanForImageClassification.from_pretrained(model_dir, config=config)

# Print model summary
print(model)



VanForImageClassification(
  (van): VanModel(
    (encoder): VanEncoder(
      (stages): ModuleList(
        (0): VanStage(
          (embeddings): VanOverlappingPatchEmbedder(
            (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
            (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (layers): Sequential(
            (0): VanLayer(
              (drop_path): Identity()
              (pre_normomalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (attention): VanSpatialAttentionLayer(
                (pre_projection): Sequential(
                  (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
                  (act): GELUActivation()
                )
                (attention_layer): VanLargeKernelAttentionLayer(
                  (attention): VanLargeKernelAttention(
                    (depth_wise): Conv2d

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=INIT_LR)
criterion = torch.nn.CrossEntropyLoss()

# Training loop
num_epochs = 1  # Change this to the number of epochs you want to run
epoch_losses = []  # To store losses for each epoch

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    train_loss = 0.0

    for batch_idx, (data, target) in enumerate(trainGenerator):
        if batch_idx >= 500: ## because taking too long currently train gen has 8k instances ,, 10 instance take 1 min
          break
        optimizer.zero_grad()  # Clear gradients

        # Convert data and target to PyTorch tensors
        data, target = torch.tensor(data), torch.tensor(target)

        # Forward pass
        output = model(data)
        logits = output.logits  # Extract the logits tensor

        # Calculate the loss
        loss = criterion(logits, target.argmax(dim=1))  # Assuming one-hot encoded targets

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        print(f"{batch_idx } Item loss: {loss.item():.4f}")
    # Calculate average loss for the epoch
    avg_loss = train_loss / len(trainGenerator)
    epoch_losses.append(avg_loss)

    #print(f"Epoch [{epoch+1}/{num_epochs}], Avg Loss: {avg_loss:.4f}") Comment because cutting the epoch smaller

# Calculate the average loss across all epochs
#total_avg_loss = sum(epoch_losses) / len(epoch_losses)
#print(f"Total Avg Loss: {total_avg_loss:.4f}")

0 Item loss: 2.1392
1 Item loss: 3.0253
2 Item loss: 2.7898
3 Item loss: 2.2817
4 Item loss: 2.5363
5 Item loss: 2.3822
6 Item loss: 2.2959
7 Item loss: 2.0358
8 Item loss: 1.9441
9 Item loss: 1.7356
10 Item loss: 2.2404
11 Item loss: 2.1858
12 Item loss: 1.9754
13 Item loss: 2.0936
14 Item loss: 1.7417
15 Item loss: 2.1481
16 Item loss: 1.9037
17 Item loss: 1.9451
18 Item loss: 1.6769
19 Item loss: 1.9284
20 Item loss: 2.1577
21 Item loss: 2.3736
22 Item loss: 2.0857
23 Item loss: 1.9604
24 Item loss: 1.9736
25 Item loss: 2.1176
26 Item loss: 1.9622
27 Item loss: 2.0456
28 Item loss: 2.0354
29 Item loss: 1.8717
30 Item loss: 1.8653
31 Item loss: 1.9657
32 Item loss: 2.0996
33 Item loss: 1.9276
34 Item loss: 1.8836
35 Item loss: 1.4752
36 Item loss: 1.8724
37 Item loss: 2.0401
38 Item loss: 1.6643
39 Item loss: 1.6835
40 Item loss: 1.6228
41 Item loss: 1.7277
42 Item loss: 1.7378
43 Item loss: 1.5320
44 Item loss: 1.6492
45 Item loss: 1.7555
46 Item loss: 1.7980
47 Item loss: 1.8041
48

In [ ]:
# Validation loop
model.eval()  # Set the model to evaluation mode
val_loss = 0.0

with torch.no_grad():
    for batch_idx, (val_data, val_target) in enumerate(valGenerator):
        if batch_idx >= 10: ## because taking too long currently train gen has 8k instances ,, 10 instance take 1 min
          break
        val_data, val_target = torch.tensor(val_data, dtype=torch.float32), torch.tensor(val_target)

        # Forward pass
        val_output = model(val_data)
        print(val_output)
        val_logits = val_output.logits  # Extract logits

        # Convert val_target to class indices (assuming val_target is one-hot encoded)
        val_target_indices = torch.argmax(val_target, dim=1)

        # Calculate the loss
        loss = criterion(val_logits, val_target_indices)
        print(f"{batch_idx } Item loss: {loss.item():.4f}")

        val_loss += loss.item()

    avg_val_loss = val_loss / (batch_idx + 1)
    print(f"Validation Loss: {avg_val_loss:.4f}")


ImageClassifierOutputWithNoAttention(loss=None, logits=tensor([[ 2.5847,  2.0831,  1.9208,  ..., -7.0604, -6.8929, -6.5956],
        [ 2.5861,  2.0855,  1.9219,  ..., -7.0643, -6.8965, -6.5990],
        [ 2.1997,  1.5959,  1.9217,  ..., -6.8397, -6.7088, -6.3900],
        ...,
        [ 1.5149,  1.4782,  2.0410,  ..., -7.5600, -7.4144, -6.9511],
        [ 2.1375,  1.6322,  2.1469,  ..., -7.3683, -7.2328, -6.8758],
        [ 2.4138,  1.8084,  1.9583,  ..., -6.9640, -6.8254, -6.5502]]), hidden_states=None)
0 Item loss: 1.7365
ImageClassifierOutputWithNoAttention(loss=None, logits=tensor([[ 2.0777,  1.3151,  1.6552,  ..., -5.9774, -5.8949, -5.6501],
        [ 2.5847,  2.0831,  1.9208,  ..., -7.0604, -6.8929, -6.5956],
        [ 2.3703,  1.9649,  2.1671,  ..., -7.9076, -7.7143, -7.3685],
        ...,
        [ 2.4588,  2.5147,  2.3950,  ..., -9.1861, -8.9676, -8.4788],
        [ 2.3633,  1.8533,  2.0857,  ..., -7.3497, -7.1910, -6.8674],
        [ 2.2443,  1.6018,  1.7131,  ..., -6.2715, -

### The Code with accuracy

In [ ]:
from sklearn.metrics import accuracy_score

# Lists to store predicted labels and true labels
predicted_labels = []
true_labels = []

model.eval()  # Ensure the model is in evaluation mode

with torch.no_grad():
    for batch_idx, (val_data, val_target) in enumerate(valGenerator):
        if batch_idx >= 100:  # Limit the number of batches for faster testing
            break
        val_data, val_target = torch.tensor(val_data, dtype=torch.float32), torch.tensor(val_target)

 # Forward pass
        val_output = model(val_data)
        print(val_output)
        val_logits = val_output.logits  # Extract logits

        # Convert val_target to class indices (assuming val_target is one-hot encoded)
        val_target_indices = torch.argmax(val_target, dim=1)


        # Store predicted labels and true labels
        predicted_labels.extend(val_logits.argmax(dim=1).tolist())
        true_labels.extend(val_target_indices.tolist())

# Calculate accuracy
print("True Labels")
print(true_labels)
print("Predicted Labels")
print(predicted_labels)
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")


ImageClassifierOutputWithNoAttention(loss=None, logits=tensor([[ 9.9815,  9.5275,  8.6731,  ..., -2.8850, -2.8240, -3.5869],
        [ 9.5343,  9.8280,  8.9305,  ..., -3.0145, -3.1930, -3.3986],
        [10.0615,  9.8087,  8.7853,  ..., -3.0025, -2.9643, -3.5958],
        ...,
        [ 8.6131,  8.4997,  7.7210,  ..., -2.5021, -2.5855, -3.2886],
        [ 8.3884,  8.4005,  7.3900,  ..., -2.4480, -2.2811, -3.0996],
        [10.3174,  9.7775,  8.7226,  ..., -3.0510, -2.8034, -3.6060]]), hidden_states=None)
ImageClassifierOutputWithNoAttention(loss=None, logits=tensor([[10.0615,  9.8087,  8.7853,  ..., -3.0025, -2.9643, -3.5958],
        [ 8.3807,  9.4796,  9.2193,  ..., -3.0281, -2.6651, -4.0091],
        [ 9.5896,  9.0674,  8.1505,  ..., -2.7579, -2.7164, -3.3273],
        ...,
        [ 8.9220,  8.7017,  8.3065,  ..., -2.6423, -2.7887, -3.2941],
        [10.4960, 10.0818,  9.2655,  ..., -3.0496, -2.8960, -3.8849],
        [ 9.7704,  9.7499,  8.9552,  ..., -2.9462, -2.8788, -3.6897]]), 

# **Xception model**

Ref: https://www.kaggle.com/hassanamin/transfer-learning-vgg16-examples-using-tensorflow#Specify-the-Model

In [ ]:
from keras.applications import Xception
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D

In [ ]:
numClasses = len(set(trainLabels.isup_grade))
weightFile = "/content/xception_weights_tf_dim_ordering_tf_kernels_notop.h5"

myModel = Sequential()
myModel.add(Xception(include_top=False, pooling='avg', weights=weightFile))
myModel.add(Dense(6, activation='softmax'))

#myModel.add(activation('softmax'))
# Say not to train first layer (Xception) model. It is already trained
myModel.layers[0].trainable = False

In [ ]:
myModel.summary()

In [ ]:
# Optimaztion function
opt = Adam(lr=INIT_LR, decay=INIT_LR / epochs)

myModel.compile(loss="binary_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

In [ ]:
H = myModel.fit(trainGenerator,
                        steps_per_epoch=10, #128taking too long
                        epochs=epochs,
                        validation_data=valGenerator,
                        validation_steps=10,  #128 taking 2 long
                        verbose=1)

In [ ]:
H_df = pd.DataFrame(H.history)
H_df[['loss', 'val_loss']].plot()
H_df[['accuracy', 'val_accuracy']].plot()

Another try
RNN


In [ ]:
from keras.applications.vgg19 import VGG19
def vgg19_model(num_classes = None):
    vgg19_weights = '../input/vgg19/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'
    #model = VGG19(weights='imagenet',include_top=False, input_shape=(224, 224, 3))
    model = VGG19(weights= vgg19_weights , include_top=False, input_shape=(224, 224, 3))
    x=Dropout(0.3)(model.output)
    x=Flatten()(x)
    x =Dense(32, activation = 'relu')(x)
    x =Dropout(0.2)(x)
    output=Dense(num_classes,activation='softmax')(x)
    model=Model(model.input,output)
    return model
vgg19_conv = vgg19_model(6)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout
# create CNN Model

model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation="relu",input_shape=(256,256,3)))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding="valid"))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation="relu",input_shape=(256,256,3)))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding="valid"))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation="relu",input_shape=(256,256,3)))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding="valid"))

model.add(Flatten())

model.add(Dense(128,activation="relu"))
# model.add(Dropout(0.05))
model.add(Dense(64,activation="relu"))
# model.add(Dropout(0.025))
model.add(Dense(1,activation="sigmoid"))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 128)      0

In [ ]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

In [ ]:
hist=model.fit(trainGenerator,epochs=10,validation_data=valGenerator)

NameError: ignored